## Chiptune Generation Project Sandbox

Notebook for trying out some code in an interactive, sandbox-like environment.

In [1]:
from modules import mcc_waves, mcc_builder

import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

sr = 44100

In [6]:
st = "16e6,16e6,32p,8e6,16c6,8e6,8g6,8p,8g,8p,8c6,16p,8g,16p,8e,16p,8a,8b,16a#,8a,16g.,16e6,16g6,8a6,16f6,8g6,8e6,16c6,16d6,8b,16p,8c6,16p,8g,16p,8e,16p,8a,8b,16a#,8a,16g.,16e6,16g6,8a6,16f6,8g6,8e6,16c6,16d6,8b,8p,16g6,16f#6,16f6,16d#6,16p,16e6,16p,16g#,16a,16c6,16p,16a,16c6,16d6,8p,16g6,16f#6,16f6,16d#6,16p,16e6,16p,16c7,16p,16c7,16c7,p,16g6,16f#6,16f6,16d#6,16p,16e6,16p,16g#,16a,16c6,16p,16a,16c6,16d6,8p,16d#6,8p,16d6,8p,16c6"
st2 = '16e6,16p,8a,16g.,16e6,16p,8g,16p,8a,8b,16p,8a,16g.,16e6,16g6,16f#6,16f6,16d#6,16p,8g,8p,8c6,16p,16g#,16a,16c6,16d6,8b,16a#,8a'
st3 = '16p,16g#,16a,16c6,16d6,8b,16p,16g#,16a,16c6,16d6,8p,16d6,8b,16a#,8a,8b,16a#,8a,8b,16a#,8a,8b,8p,8c6,16p,8a,8b,16a#,8a,16g.,16e6,16g6,16f#6,16f6,8g6,8p,16d6,8b,16a#,8a,8b,16p,16g#,16a,16c6,16p,8g,16p,8e,16p,8e,16p,8g,8p,16d6,8p,8g,16p,16c7,16c7,p,16g6,8a6,16f6,16d#6,16p,8e,16p,16g#,16a,16c6,16d6,8p,16d6,8p,8c6,16p,8c6,16p,8g,8p,16d6,8b,8p,16g6,8a6,16f6,8g6,8e6,16c6,16p,8c6,16p,16a,16c6,16d6,8b,16p,16g#'

res = mcc_waves.notes_to_waveform(st2, bpm=100, wave_function=mcc_waves.square_wave)
ipd.Audio(res, rate=sr)

In [7]:
# mid = mcc_parser.open_midi("data/castlevania.mid")

# tracks = mcc_parser.extract_midi_tracks(mid.tracks)
# track_waves = mcc_waves.create_all_track_waves(tracks, envl=True)
# result = mcc_builder.combine_tracks(track_waves)